In [1]:
#https://tensorflowkorea.gitbooks.io/tensorflow-kr/g3doc/tutorials/mnist/beginners/
#caffe -> lenet.prototext 참고
#이 코드만 있으면 구현 가능하다. 초급의 과정을 생각해가며 쭉 따라가보기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
import tensorflow as tf

#가중치 초기화

#기본def weight_variable(shape):#처음:자비아 추가 후 0으로 두고(이니시얼레이제이션 = 0) - 세번쨰 : 여기에 자비어 값 추가
#    initial = tf.truncated_normal(shape, stddev=0.1)
#    return tf.Variable(initial)
    
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#합성곱(Convolution)과 풀링(Pooling)
def conv2d(x, W):#처음:컨볼루션만
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

#첫 번째 합성곱 계층
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

W_conv1 = tf.get_variable("W1", shape=[5, 5, 1, 32], initializer=tf.contrib.layers.xavier_initializer())
#W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])#

#h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_conv1 = tf.nn.sigmoid(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#두 번째 합성곱 계층
#W_conv2 = weight_variable([5, 5, 32, 64])
W_conv2 = tf.get_variable("W2", shape=[5, 5, 32, 64], initializer=tf.contrib.layers.xavier_initializer())
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)#처음:렐루 없애고 - 두번째: 렐루만 살리고
h_pool2 = max_pool_2x2(h_conv2)

#완전 연결 계층 (Fully-Connected Layer)
#W_fc1 = weight_variable([7 * 7 * 64, 1024])
W_fc1 = tf.get_variable("Wc1", shape=[7 * 7 * 64, 1024], initializer=tf.contrib.layers.xavier_initializer())
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#드롭아웃 (Dropout)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#최종 소프트맥스 계층
#W_fc2 = weight_variable([1024, 10])
W_fc2 = tf.get_variable("W_fc2", shape=[1024, 10], initializer=tf.contrib.layers.xavier_initializer())
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

#모델의 훈련 및 평가
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(1000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
step 0, training accuracy 0.12
step 100, training accuracy 0.22
step 200, training accuracy 0.62
step 300, training accuracy 0.68
step 400, training accuracy 0.86
step 500, training accuracy 0.9
step 600, training accuracy 0.78
step 700, training accuracy 0.9
step 800, training accuracy 0.92
step 900